# imports 

In [18]:
%reset -f

# imports

In [1]:
from captum.attr import ShapleyValueSampling
from load_data import load_data
from models.model_wrappers import *
from models.train_models import *
from segmentation import *
from utils import *
import torch
import os
from tqdm import tqdm
import timeit
import sys
from torch.cuda import is_available as is_GPU_available
# device for torch
device = "cuda" if is_GPU_available() else "cpu"

# device for torch

# hyper-parameters

In [2]:

# settings
dataset_names = {'gunpoint'}    #{sys.argv[1]}
predictor_names = {'resNet'}    #{sys.argv[2]} {"randomForest", 'miniRocket', 'resNet'}
segmentation_names = [ "equal" ] #,"clasp","greedygaussian", "infogain","nnsegment"]  # {"clasp","greedygaussian", "equal", "infogain","nnsegment"} # {"clasp","greedygaussian", "equal", "infogain","nnsegment"} 
background_names =  [ "sampling", "average" , "zero"] # , "sampling",] #{"average", "zero", "sampling"}
normalization_names = {"default", "normalized"}

demo_mode = False
# demo
if demo_mode:
    dataset_names = {'gunpoint'}
    predictor_names = {"randomForest"}
    segmentation_names = { "equal"} #,'clasp'}
    background_names ={"average","sampling"} #,'sampling'}
    normalization_names = {"default", "normalized"}


# instantiate dictionaries that gonna be used in the pipeline

In [3]:
# dictionary mapping predictors to torch vs other, step necessary for Captum 
predictors = {
    'torch' : ['resNet'],
    'scikit' : ['miniRocket','randomForest','QUANT']
}
segmentation_dict = {"clasp":get_claSP_segmentation, "infogain": get_InformationGain_segmentation, "greedygaussian": get_GreedyGaussian_segmentation, "equal": get_equal_segmentation, "nnsegment": get_NNSegment_segmentation}

results = dict.fromkeys(('y_test_true', 'label_mapping', "segments", 'y_test_pred', "attributions"))
for key in results.keys():
    results[key] = dict.fromkeys(dataset_names)
    
normalization_names = normalization_names | {"default"}


# train model

In [4]:
from models.predictor_utils import load_predictor, predict_proba

for dataset_name in dataset_names:
    # init dataset
    # load data
    X_train, X_test, y_train, y_test, enc = load_data(subset='all', dataset_name=dataset_name)
    # for debugging only
    if demo_mode:
        X_test = X_test[:2]
        y_test = y_test[:2]

    n_samples, n_chs, ts_length = X_test.shape

    results['y_test_true'][dataset_name] = y_test
    results['label_mapping'][dataset_name] = enc
    results["attributions"][dataset_name] = dict.fromkeys(segmentation_names)
    results["segments"][dataset_name] = dict.fromkeys(segmentation_names)
    results["y_test_pred"][dataset_name] = dict.fromkeys(predictor_names)

    predictor_dict = dict()
    # TODO not to save if in demo mode!
    for predictor_name in predictor_names:
        if demo_mode:
            dataset_name=None

        if predictor_name=='resNet':
            # TODO rollback to normal!!!!!!!!!!!!!!
            clf = load_predictor(path="trained_models",predictor_name="resNet",dataset_name=dataset_name,device="cuda")
            preds = predict_proba(clf,samples=X_test,device="cuda")
            #clf,preds = train_ResNet(X_train, y_train, X_test, y_test, dataset_name,device=device)
        elif predictor_name=='miniRocket':
            clf,preds = train_miniRocket(X_train, y_train, X_test, y_test, dataset_name)
        elif predictor_name=="randomForest":
            clf, preds = train_randomForest(X_train, y_train, X_test, y_test, dataset_name)
        elif predictor_name=="QUANT":
            clf, preds = train_QUANT(X_train, y_train, X_test, y_test, dataset_name)
        else:
            raise ValueError("predictor not found")

        predictor_dict[predictor_name] = {"clf": clf, "preds": preds}


In [ ]:
def initialize_result_dict(X_test,predictor_names,dataset_name,segmentation_name,results):
	init_segments = np.empty((X_test.shape[0], X_test.shape[1]), dtype=object) if X_test.shape[1] > 1 else (
		np.empty(X_test.shape[0], dtype=object))
	results["segments"][dataset_name][segmentation_name] = init_segments.copy()
	results["attributions"][dataset_name][segmentation_name] = dict.fromkeys(predictor_names)
	for predictor_name in predictor_names:
		results["attributions"][dataset_name][segmentation_name][predictor_name] = dict.fromkeys(background_names)


def get_sample_info(segmentation_method, X_test,y_test,results, id, mask_list, ts_list, y_list):
	# get current sample and label
	ts, y = X_test[id], torch.tensor(y_test[id:id + 1])
	# get segment and its tensor representation
	current_segments = segmentation_method(ts)[:X_test.shape[1]]
	results['segments'][dataset_name][segmentation_name][i] = current_segments
	mask = get_feature_mask(current_segments, ts.shape[-1])
	mask_list.append(mask)
	ts = torch.tensor(ts).repeat(1, 1, 1)  # TODO use something similar to np.expand_dim?
	ts_list.append(ts)
	y_list.append(y)
	return ts,y,mask


from utils import sample_background
def get_background( background_name, results, normalization_names, X_train, n_background=50):

	results["attributions"][dataset_name][segmentation_name][predictor_name][background_name] = dict.fromkeys(
		normalization_names)
	# background data
	if background_name == "zero":
		background_dataset = torch.zeros((1,) + X_train.shape[1:])
	elif background_name == "sampling":
		background_dataset = sample_background(X_train, n_background)
	elif background_name == "average":
		background_dataset = sample_background(X_train, n_background).mean(axis=0, keepdim=True)

	return background_dataset


def get_attribution(ts, mask, background_dataset,y, results ): #    global ts, mask, background_dataset, tmp, y
    if len(background_dataset)==4:
        # in this case background is 'SAMPLING'
        # get rid of first dimension as it's always 1
        ts = ts[0] ;  mask= mask[0] ; background_dataset= background_dataset[0] ; y=y[0]
        
    if predictor_name in predictors['scikit']:
		# if using random forest flat everything
		#if predictor_name == "randomForest":
		#	ts = ts.reshape(-1, n_chs * ts_length)
		#	mask = mask.reshape(-1, n_chs * ts_length)
		#	background_dataset = background_dataset.reshape(-1, n_chs * ts_length)

        tmp = SHAP.attribute(ts, target=y, feature_mask=mask, baselines=background_dataset, additional_forward_args=clf)

    elif predictor_name in predictors['torch']:
		# if use torch make sure everything is on selected device
		ts = ts.to(device); y = y.to(device) ; mask = mask.to(device); background_dataset = background_dataset.to(device)
		tmp = SHAP.attribute(ts, target=y, feature_mask=mask, baselines=background_dataset)

	# in case of random forest 'un-flatten' result
    if predictor_name=="randomForest":
		tmp = tmp.reshape(-1,X_test.shape[1],X_test.shape[2])

	# lastly store current explanation in the data structure; if sampling store the mean
    results['attributions'][dataset_name][segmentation_name][predictor_name][background_name]["default"][i] = torch.mean(tmp, dim=0).cpu().numpy() if \
		background_name=="sampling" else tmp[0].cpu().numpy()


def get_normalized_results(normalization_names,results):
	if "normalized" in normalization_names:
		weights = np.array(list(map(
			lambda segmentation: list(map(
				lambda channel_segemnts: lengths_to_weights(change_points_to_lengths(channel_segemnts, X_train.shape[-1])),
				segmentation)),
			results["segments"][dataset_name][segmentation_name])))

		results['attributions'][dataset_name][segmentation_name][predictor_name][background_name]["normalized"] = \
			results['attributions'][dataset_name][segmentation_name][predictor_name][background_name]["default"] * weights
	if "default" not in normalization_names:
		del results['attributions'][dataset_name][segmentation_name][predictor_name][background_name]["default"]


In [5]:
def initialize_result_dict(X_test,predictor_names,dataset_name,segmentation_name,results):
    
    init_segments = np.empty((X_test.shape[0], X_test.shape[1]), dtype=object) if X_test.shape[1] > 1 else ( np.empty(X_test.shape[0], dtype=object))
    results["segments"][dataset_name][segmentation_name] = init_segments.copy()
    results["attributions"][dataset_name][segmentation_name] = dict.fromkeys(predictor_names)
    for predictor_name in predictor_names:
        results["attributions"][dataset_name][segmentation_name][predictor_name] = dict.fromkeys(background_names)

In [6]:
def get_sample_info(segmentation_method, mask_list, ts_list, y_list, sample):
    
    # get current sample and label
    ts, y = torch.tensor(sample[0] ) , torch.tensor((sample[1]))
    # get segment and its tensor representation
    current_segments = segmentation_method(ts[0])[:X_test.shape[1]]
    mask = get_feature_mask(current_segments, ts.shape[-1])
    
    # append any relevant information into the correct list
    mask_list.append(mask)
    ts_list.append(ts)
    y_list.append(y)
    return ts,y,mask, current_segments


In [7]:
from utils import sample_background

def get_background( background_name, X_train, n_background=50):

    # background data
    if background_name == "zero":
        background_dataset = torch.zeros((1,) + X_train.shape[1:])
    elif background_name == "sampling":
        background_dataset = sample_background(X_train, n_background)
    elif background_name == "average":
        background_dataset = sample_background(X_train, n_background).mean(axis=0, keepdim=True)

    return background_dataset


In [8]:
def get_attribution(explainer, ts, mask, background_dataset,y, sampling ): 
    
    if sampling:
        # get rid of first dimension as it's always 1
        # TODO try to flatten multiple singles "50 samples" into a 3D dataset and get the performances of that
        ts = ts[0] ;  mask= mask[0] ; y=y[0]

    if predictor_name in predictors['scikit']:
        
        tmp = explainer.attribute(ts, target=y, feature_mask=mask, baselines=background_dataset, additional_forward_args=clf)

    elif predictor_name in predictors['torch']:
        # if use torch make sure everything is on selected device
        ts = ts.to(device); y = y.to(device) ; mask = mask.to(device); background_dataset = background_dataset.to(device)
        tmp = explainer.attribute(ts, target=y, feature_mask=mask, baselines=background_dataset)

    # in case of random forest 'un-flatten' result
    if predictor_name=="randomForest":
        tmp = tmp.reshape(-1,X_test.shape[1],X_test.shape[2])

    # lastly store current explanation in the data structure; if sampling store the mean
    saliency_map = torch.mean(tmp, dim=0).cpu().numpy() if sampling else tmp.cpu().numpy()
    return saliency_map


In [9]:
def store_results(table, segmentation_name, normalization_names, current_results, start):
    
    # store "default" result
    n_results = current_results.shape[0]
    if 'default' in normalization_names:
        table['default'][start: (start+n_results) ] = current_results

    
    # and normalized result
    # TODO to be improved!
    if "normalized" in normalization_names:
        weights = np.array(list(map(
            lambda segmentation: list(map(
                lambda channel_segemnts: lengths_to_weights(change_points_to_lengths(channel_segemnts, X_train.shape[-1])),
                segmentation)),
            results["segments"][dataset_name][segmentation_name][start: (start+n_results) ]  )))
        
        table['normalized'][start: (start+n_results) ]  = current_results * weights
    

# DO WE WANT A STATIC BACKGROUND????????????????????????????????????????????????

In [10]:
from models.SHAP_dataloader import SHAP_dataloader
from torch.utils.data import DataLoader

starttime = timeit.default_timer()

with torch.no_grad():
    for dataset_name in dataset_names:
        for predictor_name in predictor_names:
            # copy current predictor prediction into the 'results' dictionary to be dumped
            results['y_test_pred'][dataset_name][predictor_name] = predictor_dict[predictor_name]["preds"]
        for segmentation_name in segmentation_names:
            
            # initialize part of the dictionary
            initialize_result_dict(X_test,predictor_names,dataset_name,segmentation_name,results)
            segmentation_method = segmentation_dict[segmentation_name]
            ts_list = []
            mask_list = []
            y_list = []
            
            # first run the segmentation
            for i in range(n_samples) : 
                ts,y,mask, current_segment = get_sample_info( segmentation_method, mask_list, ts_list, y_list , sample=( X_test[i:i+1], y_test[i:i + 1])  )
                results['segments'][dataset_name][segmentation_name][i] = current_segment

            for background_name in background_names:

                # get the background for the current data
                results["attributions"][dataset_name][segmentation_name][predictor_name][background_name] = dict.fromkeys(normalization_names)
                background_dataset = get_background( background_name, X_train)

                for predictor_name in predictor_names:
                    # get classifier and initialize attributions
                    clf = predictor_dict[predictor_name]["clf"]
                    init_attributions = np.zeros(X_test.shape, dtype=np.float32)
                    for normalization_name in normalization_names:
                        results['attributions'][dataset_name][segmentation_name][predictor_name][background_name][normalization_name] = init_attributions.copy()

                    # instantiate SHAP explainer                    
                    SHAP = ShapleyValueSampling(clf) if predictor_name in predictors['torch'] else ShapleyValueSampling(forward_classification)
                    
                    # prepare for batch computation
                    batch_size = 1 if background_name=='sampling' else 50 #HARD CODEEEEEEED!!!!!!!!!!!!!!!
                    data_loader = DataLoader( SHAP_dataloader(ts_list,y_list,mask_list, background_dim=background_dataset.shape[0] ) ,  batch_size=batch_size )
                    
                    # computation loop
                    current_idx = 0
                    with tqdm(total=len(ts_list)) as pbar:
                        for (ts,y,mask) in data_loader:
                            
                            current_results = get_attribution( SHAP, ts,mask,background_dataset,y, 
                                    sampling= (background_name=='sampling') )
                            
                            store_results(table=results['attributions'][dataset_name][segmentation_name][predictor_name][background_name], segmentation_name =segmentation_name, normalization_names=normalization_names,current_results=current_results, start=current_idx)
                            
                            pbar.update(batch_size) ; current_idx+=batch_size

                    pbar.close()
                    
print("elapsed time", ( timeit.default_timer() -starttime ) )


100%|██████████| 150/150 [00:03<00:00, 43.32it/s]

elapsed time 178.1567400419999


In [11]:
ttmp =  results['attributions'][dataset_name][segmentation_name][predictor_name]
for k in ttmp.keys():
    print( "\n\n", k)
    for norm in [ 'default', 'normalized']:
        ttttmp = np.sum(np.abs(ttmp[k][norm]) , axis= -1)
        print( ttttmp , ttttmp.shape )
    



 sampling
[[17.213726 ]
 [22.527115 ]
 [17.002495 ]
 [16.803356 ]
 [25.36367  ]
 [24.160076 ]
 [12.436918 ]
 [19.953375 ]
 [16.504854 ]
 [23.993069 ]
 [23.67686  ]
 [16.177044 ]
 [22.404737 ]
 [17.047768 ]
 [16.558947 ]
 [25.284199 ]
 [25.200077 ]
 [18.603773 ]
 [14.167227 ]
 [16.414976 ]
 [17.008774 ]
 [20.796623 ]
 [22.515518 ]
 [26.407187 ]
 [16.736118 ]
 [18.652033 ]
 [17.359257 ]
 [17.492317 ]
 [14.124632 ]
 [20.839365 ]
 [22.777634 ]
 [21.500404 ]
 [21.97852  ]
 [24.860481 ]
 [23.052101 ]
 [20.434687 ]
 [21.6032   ]
 [24.534752 ]
 [16.726856 ]
 [12.483172 ]
 [24.089485 ]
 [20.312641 ]
 [23.161274 ]
 [25.39692  ]
 [25.620148 ]
 [16.1157   ]
 [21.420471 ]
 [17.027956 ]
 [17.318619 ]
 [20.468327 ]
 [21.325954 ]
 [19.525883 ]
 [23.300411 ]
 [18.821102 ]
 [23.989296 ]
 [17.018208 ]
 [20.734177 ]
 [19.868172 ]
 [22.461853 ]
 [ 4.8214817]
 [17.002562 ]
 [16.604675 ]
 [16.978664 ]
 [18.95317  ]
 [21.343739 ]
 [22.129608 ]
 [16.661108 ]
 [17.645382 ]
 [22.324991 ]
 [24.946224 ]
 [22.496

In [39]:
# dump result to disk
if not demo_mode:
    file_name = "_".join( ("all_results",dataset_name,predictor_name) )
else:
	file_name = "_".join( ("all_results_DEMO_",dataset_name,predictor_name) )
file_path = os.path.join("attributions", file_name)
np.save( file_path, results )

In [26]:
results["y_test_pred"]

{'gunpoint': {'QUANT': array([[0.985, 0.015],
         [0.015, 0.985]])}}